# Извлечение именованных сущностей.

Именованные сущности - общий термин, который используют для обозначения какого-то множества слов (словосочетаний, последовательностей символов), которые представляют какой-то особый инетерес в контексте решаемой практической задачи и которые нужно отделить от остальных слов. Это могут быть какие-то стандартные вещи: имена, фамилии, названия организаций, локации, денежные суммы, даты, номера телефонов, а могут быть и более специфичные: статьи кодексов, белки, симптомы, товары.


В многих случаях случаях можно написать регулярку и решиьть задачу на приемлимом уровне (например собрать все номера телефонов совсем несложно). Но когда сущности выраются нестандартным образом, постоянно расширяются и меняются по формам, задача становится очень сложной.

Стандартные сущности ещё можно извлекать с помощью готовых инструментов.

Для **английского** удобно использовать **spacy**. Там сразу извлекаются сущности с хорошим качеством.

Для **русского** (если не хочется ничего делать) можно использовать **.tag** из **pymorphy**.

In [0]:
!pip install pymorphy2

In [0]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [3]:
p = morph.parse('Михаил')[0].tag
print('Тэги - ', p)
print('Name' in p) #тэг имени

Тэги -  NOUN,anim,masc,Name sing,nomn
True


In [4]:
p = morph.parse('Иванов')[0].tag
print('Тэги - ', p)
print('Surn' in p) #тэг фамилии

Тэги -  NOUN,anim,masc,Sgtm,Surn sing,nomn
True


In [5]:
p = morph.parse('Петрович')[0].tag
print('Тэги - ', p)
print('Patr' in p) #тэг отчества

Тэги -  NOUN,anim,masc,Patr sing,nomn
True


In [6]:
p = morph.parse('Москва')[0].tag
print('Тэги - ', p)
print('Geox' in p) #тэг локация

Тэги -  NOUN,inan,femn,Sgtm,Geox sing,nomn
True


In [7]:
p = morph.parse('Яндекс')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Orgn sing,nomn
True


In [11]:
p = morph.parse('')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  UNKN
False


Работает не очень хорошо, но все равно лучше, чем ничего. Рядом стоящие слова одного тэга можно склеить в один. Или сначала собрать нграмы и если какое-то одно слово в нграмме принадлежит к какому-то типу сущности, то распространить его на весь нграм.

Другой инструмент - **natasha** (https://github.com/natasha/natasha)

Она основана на **парсере yargy** https://github.com/natasha/yargy и представляет собой набор готовых правил для извлечения некоторых сущностей.

In [0]:
!pip install natasha

In [0]:
from natasha import (NamesExtractor,
                     SimpleNamesExtractor,
                     PersonExtractor,
                     LocationExtractor,
                     AddressExtractor,
                     OrganisationExtractor,
                     DatesExtractor,
                     MoneyExtractor,
                     MoneyRateExtractor,
                     MoneyRangeExtractor)

from natasha.markup import (show_markup_notebook as show_markup,
                            format_json)

In [17]:
text = 'Влад Веселов. Петрович. Алиса. Студия Артемия Лебедева'

extractor_per = NamesExtractor()
matches = extractor_per(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
print(format_json(facts))

[
  {
    "first": "влад",
    "last": "веселов"
  },
  {
    "first": "артемий",
    "last": "лебедев"
  }
]


In [16]:
text = 'Влад Веселов. Петрович. Алиса. Студия Артемия Лебедева'

extractor_per = PersonExtractor()
matches = extractor_per(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [18]:
text = 'Более того в Москве, в районе Строгино. На реке Оке. В германии'

extractor_loc = LocationExtractor()
matches = extractor_loc(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
print(format_json(facts))

[
  {
    "name": "то"
  },
  {
    "name": "москва"
  },
  {
    "name": "строгино"
  },
  {
    "name": "ока"
  },
  {
    "name": "германия"
  }
]


In [0]:
text = 'ФСБ. Московский государственный университет. Высшая школа экономика. ВШЭ. Mail.ru'

extractor_org = OrganisationExtractor()
matches = extractor_org(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [0]:
text = 'С 2015 по 2017 год. 16 апреля 1993 года. В субботу. 23.04.18'

extractor_date = DatesExtractor()
matches = extractor_date(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [0]:
text = "Он заплатил ему 300 рублей."

extractor_money = MoneyExtractor()
matches = extractor_money(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

Ещё есть **томита-парсер**, но с ним очень тяжело работать (никакого развития, скудная документация, закрытый код, никакого сообщества) https://tech.yandex.ru/tomita/

Если нужно улучшить выделение стандартныйх типов или научиться извлекать специфичные сущности, нужно **писать правила в Yargy**. 

Возьмем тексты объявлений на авито по категории Консоли и попробуем научиться выделять названия приставок Xbox и Playstation.

In [0]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

Посмотрим на данные.

In [0]:
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/%E8%AF%BE%E5%A0%82%E5%86%85%E5%AE%B9/191219ner/pristavki.csv

In [0]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [22]:
data.head(10)

,text
0,"Состояние: Отличное для возраста приставки./\nИгры отдам вместе с приставкой./\nОстальное по телефону, либо при осмотре."
1,"+5 игар фифа15,16 мартол комбат 9, гта5,асассин 4, и три диска Фифв 13 гта 4 видмак3"
2,"Продам в отличном состоянии , в комплекте проводной и беспроводной геймпады (зарядное устройство для беспроводного), гарнитура , два диска . Вообщем полный комплект, но без Kinect."
3,"Продам PS3, 2 беспроводных джойстика в комплекте. Коллекция игр будет естественно бонусом) на фото не хватает игры Mortal combat vs D.C. Джойстики не залипают,можете проверить при покупке. Обмен не интересен, так как подарили Ps4) небольшой торг )"
4,"Продаю в связи с ненадобностью.Купила для мужа ,играть не стал.Состояние идеальное .Любые проверки.Предложение идеально до 21 апреля"
5,"Код не активировал,обмен"
6,Возможен обмен
7,"Xbox в отличном состоянии, 2 диска с играми /\n/\nПоддержка носителей: CD, DVD, флэшка./\n/\nWi-Fi (802.11)/\nBluetooth. Цвет: черный"
8,"Обмен интересен на FIFA16,17 или на GTA5"
9,"Джойстик в стиле PS1 для игровых приставок Dendy. Цвет — белый./\n /\nНаш магазин находятся по адресу: /\n*** г.Зеленодольск, Татарстан 10 2 этаж, График работы магазина: с 10:00-20:00, Без выходных/\n /\n❀❀❀❀❀Почему наши покупатели всегда довольны❀❀❀❀❀/\n✔ЗОЛОТАЯ МЕДАЛЬ от авито в графе продавец!/\n✔нас более 50 магазинов по всей России, что позволяет нам держать самые низкие цены за счет прямых поставок с фабрик/\n✔отличное качество и огромный ассортимент!/\n✔постоянные супер-выгодные для покупателей акции/\n✔подарки за покупки от 2000 руб/\n✔заводская гарантия 6 месяцев/\n✔все товары в наличии (если что-то закончилось - то, как правило, находится в пути)/\n✔возле магазинов имеются парковки!/\n✔чтобы нас не потерять - добавляйтесь в группу ""Вконтакте"" и Инстаграм ""Мир удивительных товаров""!/\n /\n✔Работаем ЕЖЕДНЕВНО!/\n /\n☎ ЗВОНИТЕ ПРЯМО СЕЙЧАС и получите консультацию!/\n______________________/\nБудем рады видеть Вас в числе наших довольных покупателей!/\nЕЩЕ БОЛЬШЕ НУЖНЫХ ВАМ ТОВАРОВ НА ФОТОГРАФИЯХ НИЖЕ/\n↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓"


Давайте попробуем сначала полные варианты: Xbox 360, Xbox one, Playstation 1,2,3,4.

In [0]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

Для начала определим саму сущность, которую будет извлекать. Сущность будет называться Pristavka и неё будет два атрибута: название и версия. Делается это вот так:

In [0]:

Pristavka = fact(
    'Pristavka',
    ['name', 'model']
)

(Чуть позже станет понятнее зачем так делать)

Теперь сделаем правило для иксбокса. Просто напишем несколько вариантов написания Xbox и версий.
Такой список называется Газзетир. В Yargy его удобно задавать через morph_pipeline. Туда можно написать слова или последовательности, а он их нормализует, чтобы потом сопоставлять с текстом.


В итоге у нас два газзетира. Мы оборачиваем их в правило (rule). Работать это будет примерно как регулярка. 

In [0]:
Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name), # то, что сматчиться будет в атрибуте name
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model)) # то, что сматчиться будет в атрибуте version


Правило для плейстешена будет аналогичное.

In [0]:
PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model)    
    )

Теперь сделаем общее правило, которое будет искать плейстешены или иксбоксы и извлекать их как сущность типа Pristavka.

In [0]:
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

Попробуем сматчить по текстам.

In [0]:
matches = []

for sent in data.text[:100]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [29]:
for m in matches:
    print(m.name, m.model)

PS 3
Ps 4
PS 1
PS 3
PlayStation 3
PS 3
Xbox 360
Playstation 3
Ps 4
ps 2
Xbox 360
PS 4
PS 3
PS 4
PS 3
Xbox 360
Xbox One
PS 4
Xbox 360
Xbox One
PlayStation 3
xbox 360
PS 4
Xbox 360
Playstation 3
Xbox 360
Playstation 4
Xbox One
ps 4
ps 4
Xbox 360
XBox 360
PS 3
XBox 360
PS 3
Ps 3


У приставок ещё могут быть подификации вроде Slim, X, S и т.д, но не всегда. Чтобы учесть это можно добавить ещё одно правило с .optional() на конце.

In [0]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model),
    morph_pipeline(['s', 'x', 'e']).interpretation(Pristavka.version).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model),
    morph_pipeline(['Slim', 'SuperSlim', 'слим']).interpretation(Pristavka.version).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [0]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [32]:
for m in matches:
    print(m.name, m.model)

PS 3
Ps 4
PS 1
PS 3
PlayStation 3
PS 3
Xbox 360
Playstation 3
Ps 4
ps 2
Xbox 360
PS 4
PS 3
PS 4
PS 3
Xbox 360
Xbox One
PS 4
Xbox 360
Xbox One
PlayStation 3
xbox 360
PS 4
Xbox 360
Playstation 3
Xbox 360
Playstation 4
Xbox One
ps 4
ps 4
Xbox 360
XBox 360
PS 3
XBox 360
PS 3
Ps 3
PS 3
PS 3
PS 4
ps 4
PS 4
xbox 360
X box One
xbox 360
PS 3
XBOX 360
XBOX 360
Playstation 3
PS 2
PlayStation 3
PlayStation 3
PlayStation 3
Playstation 3
Xbox 360
Playstation 4
Xbox One
XBOX ONE
PS 2
PS 4
Xbox One
PS 4
PS 4
playstation 3
PS 3
Xbox one
PS 4


Все работает как и должно.

Хорошо бы ещё приводить все варианты названия к нормальному виду. Вот как это можно сделать.

Если можно заменить весь газзетир на какое-то одно слово, то можно просто добавить в интерпретации значение .сonst('something')

In [0]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name.const('Xbox')),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model),
    morph_pipeline(['s', 'x', 'e']).interpretation(Pristavka.version).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name.const('Playstation')),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model),
    morph_pipeline(['Slim', 'SuperSlim', 'слим']).interpretation(Pristavka.version).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [0]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [0]:
for m in matches:
    print(m.name, m.model)

Playstation 3
Playstation 4
Playstation 1
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 3
Playstation 4
Playstation 2
Xbox 360
Playstation 4
Playstation 3
Playstation 4
Playstation 3
Xbox 360
Xbox One
Playstation 4
Xbox 360
Xbox One
Playstation 3
Xbox 360
Playstation 4
Xbox 360
Playstation 3
Playstation 4
Xbox 360
Xbox One
Playstation 4
Playstation 4
Xbox 360
Playstation 3
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Playstation 4
Playstation 4
Xbox 360
Xbox One
Xbox 360
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 2
Playstation 3
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Xbox 360
Xbox One
Xbox ONE
Playstation 2
Playstation 4
Xbox One
Playstation 4
Playstation 4
Playstation 3
Playstation 3
Xbox one
Playstation 4


В общем случае к нормальной форме приводит .normalized() добавленный к атрибуту в интерпретации. Но он работает через pymorphy2, а он работает с русским языком. Поэтому в нашем случае его можно использовать только для привода к нижнему регистру.

Когда у каждого и слов в газзетире есть своя нормальная форма, можно сделать газзетир словарём, где ключи это нужные формы, а значения - нормальные формы. А в интерпретации к .normalized() добавить .custom() и через него дергать нужную правильную форму.

In [0]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

VERSIONS = {
    'super slim': 'SuperSlim',
    'superslim': 'SuperSlim',
    'slim': 'Slim',
    'fat': 'Fat',
    'pro': 'PRO',
    'vita': 'VITA'
}

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name.const('Xbox')),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model.normalized()),
    in_caseless('sxe').interpretation(Pristavka.version.normalized()).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name.const('Playstation')),
    # для простоты можно написать вот так
    in_('1234').interpretation(Pristavka.model.normalized()),
    #изменения вот тут                                                      вот тут дергаем правильную форму
    caseless_pipeline(VERSIONS).interpretation(Pristavka.version.normalized().custom(VERSIONS.get)).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [0]:
list(parser.findall('Playstation 3 SLIM'))[0].fact

Pristavka(name='Playstation',
          model='3',
          version='Slim')

In [0]:
list(parser.findall('XbOx 360 X'))[0].fact

Pristavka(name='Xbox',
          model='360',
          version='x')

In [0]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [0]:
for m in matches:
    print(m.name, m.model)

Playstation 3
Playstation 4
Playstation 1
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 3
Playstation 4
Playstation 2
Xbox 360
Playstation 4
Playstation 3
Playstation 4
Playstation 3
Xbox 360
Xbox one
Playstation 4
Xbox 360
Xbox one
Playstation 3
Xbox 360
Playstation 4
Xbox 360
Playstation 3
Playstation 4
Xbox 360
Xbox one
Playstation 4
Playstation 4
Xbox 360
Playstation 3
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Playstation 4
Playstation 4
Xbox 360
Xbox one
Xbox 360
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 2
Playstation 3
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Xbox 360
Xbox one
Xbox one
Playstation 2
Playstation 4
Xbox one
Playstation 4
Playstation 4
Playstation 3
Playstation 3
Xbox one
Playstation 4


Улучшения для этого типа сущностей довольно очевидные и простые - нужно расширять газзетиры, можно добавить другие приставки вроде денди и нинтендо.

Для сущностей, которые нельзя перечислить словарём, нужны контекстные правила. В целом они не сильно сложнее того, что мы сейчас написали. Возможно вам понадобятся другие полезные штуки из Yargy (предикат - and_, учёт грамматической информации - gram('NOUN'), согласование по роду, числу и падежу - gnc).

Учить сразу все не имеет смысла. Возьмите практическую задачу, документацию к Yargy (https://yargy.readthedocs.io/ru/latest/index.html) или примеры (https://github.com/natasha/yargy-examples) и разбирайте только то, что вам нужно. 